<a href="https://colab.research.google.com/github/alessandranerylima/data_science/blob/main/Predi%C3%A7%C3%A3o_PLD_hor%C3%A1rio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **1. Descrição Geral do Problema**

O problema é uma tarefa de regressão simples. Com base nos dados reais divulgados pela CCEE no período de 17/04/2018 a 07/03/2022. Com base no histórico de preços (PLD- Preço de Liquidação Diária) a proposta é ajustar um modelo de previsão para ser capaz de prever o PLD horário uma HORA a frente.

# **2. Carregando os Dados**

**2.1 Importando as bibliotecas necessárias**

Importação as bibliotecas fundamentais de Python para este notebook.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import plotly.graph_objs as go

In [ ]:
import tensorflow as tf
from tensorflow import keras

print('tensorflow:      versão', tf.__version__)
print('keras integrada: versão', keras.__version__)

tensorflow:      versão 2.8.0
keras integrada: versão 2.8.0


### **2.1 Conjunto de dados**

In [ ]:
# Carregando dados extraídos da CCEE - PLD janeiro a dezembro/2021.

pld_horarioCCEE = pd.read_excel('/content/Historico_do_Preco_Horario_-_17_de_abril_de_2018_a_7_de_marco_de_2022.xlsx')

In [ ]:
# Exibindo as primeiras linhas do Dataframe.

pld_horarioCCEE.head(-1)

,Hora,Submercado,2018-04-17 00:00:00,2018-04-18 00:00:00,2018-04-19 00:00:00,2018-04-20 00:00:00,2018-04-21 00:00:00,2018-04-25 00:00:00,2018-04-26 00:00:00,2018-04-27 00:00:00,...,2022-02-26 00:00:00,2022-02-27 00:00:00,2022-02-28 00:00:00,2022-03-01 00:00:00,2022-03-02 00:00:00,2022-03-03 00:00:00,2022-03-04 00:00:00,2022-03-05 00:00:00,2022-03-06 00:00:00,2022-03-07 00:00:00
0,0,SUDESTE,40.16,40.16,40.16,112.06,40.16,40.16,130.12,129.33,...,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7
1,0,SUL,40.16,40.16,40.16,112.10,40.16,40.16,130.16,129.37,...,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7
2,0,NORDESTE,40.16,40.16,40.16,111.69,40.16,40.16,130.01,129.23,...,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7
3,0,NORTE,40.16,40.16,40.16,111.65,40.16,40.16,129.97,129.19,...,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7
4,1,SUDESTE,40.16,40.16,40.16,112.04,40.16,40.16,130.10,129.31,...,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,22,NORDESTE,136.53,117.05,117.44,132.97,40.16,138.36,136.90,155.86,...,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7
91,22,NORTE,118.29,117.01,117.40,132.93,40.16,138.32,136.86,155.82,...,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7
92,23,SUDESTE,114.26,114.86,116.44,117.70,40.16,136.68,136.86,134.42,...,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7
93,23,SUL,114.26,114.86,116.44,117.72,40.16,136.72,136.86,134.44,...,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7,55.7


In [ ]:
pld_horarioCCEE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Columns: 1409 entries, Hora to 2022-03-07 00:00:00
dtypes: float64(1407), int64(1), object(1)
memory usage: 1.0+ MB


In [ ]:
pld_horarioCCEE.describe()

,Hora,2018-04-17 00:00:00,2018-04-18 00:00:00,2018-04-19 00:00:00,2018-04-20 00:00:00,2018-04-21 00:00:00,2018-04-25 00:00:00,2018-04-26 00:00:00,2018-04-27 00:00:00,2018-04-28 00:00:00,...,2022-02-26 00:00:00,2022-02-27 00:00:00,2022-02-28 00:00:00,2022-03-01 00:00:00,2022-03-02 00:00:00,2022-03-03 00:00:00,2022-03-04 00:00:00,2022-03-05 00:00:00,2022-03-06 00:00:00,2022-03-07 00:00:00
count,96.000000,96.000000,96.000000,96.000000,96.000000,9.600000e+01,96.000000,96.000000,96.000000,96.000000,...,9.600000e+01,9.600000e+01,9.600000e+01,9.600000e+01,9.600000e+01,9.600000e+01,9.600000e+01,9.600000e+01,9.600000e+01,9.600000e+01
mean,11.500000,98.360104,95.634062,93.893646,116.567708,4.016000e+01,110.605625,133.890313,134.568750,211.575208,...,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01
std,6.958524,43.530774,35.807985,36.484077,17.272529,4.999909e-14,43.593304,14.224941,21.300348,14.051997,...,9.999817e-14,9.999817e-14,9.999817e-14,9.999817e-14,9.999817e-14,9.999817e-14,9.999817e-14,9.999817e-14,9.999817e-14,9.999817e-14
min,0.000000,40.160000,40.160000,40.160000,40.160000,4.016000e+01,40.160000,40.160000,40.160000,167.360000,...,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01
25%,5.750000,40.160000,40.160000,40.160000,112.075000,4.016000e+01,40.160000,130.155000,129.345000,195.840000,...,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01
50%,11.500000,119.070000,118.215000,118.680000,120.900000,4.016000e+01,138.465000,137.495000,140.325000,220.105000,...,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01
75%,17.250000,121.435000,119.330000,119.245000,121.140000,4.016000e+01,139.000000,138.605000,140.840000,220.852500,...,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01
max,23.000000,201.770000,119.880000,119.330000,139.600000,4.016000e+01,139.290000,139.510000,161.340000,225.600000,...,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01,5.570000e+01


Análise das colunas que contém poucos valores, chamadas de categóricas, para identificar as classes presentes.

In [ ]:
pld_horarioCCEE['Submercado'].value_counts().sort_index()

NORDESTE    24
NORTE       24
SUDESTE     24
SUL         24
Name: Submercado, dtype: int64

In [ ]:
pld_horarioCCEE['Hora']=pld_horarioCCEE['Hora'].astype('str')
pld_horarioCCEE['Hora'].value_counts()

0     4
1     4
22    4
21    4
20    4
19    4
18    4
17    4
16    4
15    4
14    4
13    4
12    4
11    4
10    4
9     4
8     4
7     4
6     4
5     4
4     4
3     4
2     4
23    4
Name: Hora, dtype: int64

In [ ]:
pld_horarioCCEE.shape

(96, 1409)

### **2.2 Reorganização dos dados**

Abaixo faremos a separação do dataset em quatro séries de dados, uma para cada submercado. Assim cada série terá um grade dataframe com um tempo (timestep) para cada linha e um único atributo (o preço naquele instante).


In [ ]:
series = pld_horarioCCEE.set_index('Submercado')
s_sudeste= series.loc[['SUDESTE']]
s_sul= series.loc[['SUL']]
s_nordeste= series.loc[['NORDESTE']]
s_norte= series.loc[['NORTE']]

In [ ]:
# mantém todas menos a última linha
s_sudeste= s_sudeste.iloc[0:24]
s_sul = s_sul.iloc[0:24]
s_nordeste = s_nordeste.iloc[0:24]
s_norte = s_norte.iloc[0:24]

In [ ]:
# reorganiza cotações por hora e dia (em colunas separadas) em um NOVO dataframe (a coluna 'y' é a cotação)
c_sudeste = s_sudeste.melt(id_vars=['Hora'], var_name='Dia', value_name='y')
c_sudeste.info

<bound method DataFrame.info of       Hora        Dia      y
0        0 2018-04-17  40.16
1        1 2018-04-17  40.16
2        2 2018-04-17  40.16
3        3 2018-04-17  40.16
4        4 2018-04-17  40.16
...    ...        ...    ...
33763   19 2022-03-07  55.70
33764   20 2022-03-07  55.70
33765   21 2022-03-07  55.70
33766   22 2022-03-07  55.70
33767   23 2022-03-07  55.70

[33768 rows x 3 columns]>

In [ ]:
# cria nova coluna combinando dia e hora
c_sudeste['ds'] = pd.to_datetime(c_sudeste['Dia'].map(lambda d: str(d.date())) + ' ' + c_sudeste['Hora'].map(str) + ':00')
c_sudeste.head(24)

,Hora,Dia,y,ds
0,0,2018-04-17,40.16,2018-04-17 00:00:00
1,1,2018-04-17,40.16,2018-04-17 01:00:00
2,2,2018-04-17,40.16,2018-04-17 02:00:00
3,3,2018-04-17,40.16,2018-04-17 03:00:00
4,4,2018-04-17,40.16,2018-04-17 04:00:00
5,5,2018-04-17,40.16,2018-04-17 05:00:00
6,6,2018-04-17,40.16,2018-04-17 06:00:00
7,7,2018-04-17,40.16,2018-04-17 07:00:00
8,8,2018-04-17,116.86,2018-04-17 08:00:00
9,9,2018-04-17,119.00,2018-04-17 09:00:00


In [ ]:
# define 'ds' como índice (é um timestamp) e remove colunas 'Dia' e 'Hora'
c_sudeste.drop(columns=['Dia', 'Hora'], inplace=True)
c_sudeste.head(4)

,y,ds
0,40.16,2018-04-17 00:00:00
1,40.16,2018-04-17 01:00:00
2,40.16,2018-04-17 02:00:00
3,40.16,2018-04-17 03:00:00


In [ ]:
c_sudeste.to_csv('c_sudeste.csv')